# RAG с използване на Azure AI Agent Service и Semantic Kernel

Този кодов фрагмент демонстрира как да създадете и управлявате Azure AI агент за генериране с подобрено извличане (RAG), използвайки `Azure AI Agent Service` и `Semantic Kernel`. Агентът обработва потребителски запитвания въз основа на извлечения контекст и предоставя точни отговори съответно.


Корекция на версията на SQLite  
Ако срещнете грешката:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Премахнете коментара от този блок код в началото на вашия ноутбук:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Импортиране на пакети
Следният код импортира необходимите пакети:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Генериране с допълнителна информация чрез извличане с Semantic Kernel и Azure AI Agent Service

Този пример демонстрира как да използвате **Azure AI Agent Service**, за да извършите **генериране с допълнителна информация чрез извличане (RAG)**, като комбинирате езиков модел със специфичен за домейна контекст от качен документ.

### Как работи

1. **Качване на документ**: Markdown файл (document.md), съдържащ информация (застрахователната политика на Contoso за пътувания), се качва в услугата за агенти.

2. **Създаване на векторно хранилище**: Документът се индексира във векторно хранилище, за да се позволи семантично търсене в съдържанието му.

3. **Конфигуриране на агент**: Агентът се създава с модела `gpt-4o` и следните строги инструкции:
   - Да отговаря само на въпроси, базирани на извлечено съдържание от документа.
   - Да отказва да отговаря, ако въпросът е извън обхвата на документа.

4. **Интеграция на инструмента за търсене в файлове**: `FileSearchTool` се регистрира към агента, което позволява на модела да търси и извлича релевантни откъси от индексирания документ по време на обработката.

5. **Взаимодействие с потребителя**: Потребителите могат да задават въпроси. Ако в документа се намери релевантна информация, агентът генерира обоснован отговор.  
   Ако не, агентът изрично отговаря, че документът не съдържа достатъчно информация.


### Основна функция

```python
def main():
    # Тази функция е началната точка на програмата
    print("Добре дошли в нашата програма!")
    initialize()
    run_application()
```

### Инициализация

```python
def initialize():
    # Тук се извършва първоначалната настройка
    print("Инициализация на системата...")
    load_config()
    setup_environment()
```

### Зареждане на конфигурация

```python
def load_config():
    # Зареждане на конфигурационните файлове
    print("Зареждане на конфигурация...")
    config = read_file("config.json")
    parse_config(config)
```

### Настройка на средата

```python
def setup_environment():
    # Настройка на работната среда
    print("Настройка на средата...")
    check_dependencies()
    prepare_resources()
```

### Стартиране на приложението

```python
def run_application():
    # Основната логика на приложението
    print("Стартиране на приложението...")
    while not is_exit_condition_met():
        process_user_input()
        update_state()
```

### Проверка на зависимости

```python
def check_dependencies():
    # Проверка дали всички зависимости са налични
    print("Проверка на зависимости...")
    if not dependencies_installed():
        print("Липсващи зависимости. Моля, инсталирайте необходимите пакети.")
        exit()
```

### Подготовка на ресурси

```python
def prepare_resources():
    # Подготовка на необходимите ресурси
    print("Подготовка на ресурси...")
    allocate_memory()
    initialize_database()
```

### Обработка на потребителски вход

```python
def process_user_input():
    # Обработка на входа от потребителя
    print("Моля, въведете команда:")
    user_input = input()
    handle_command(user_input)
```

### Актуализация на състоянието

```python
def update_state():
    # Актуализация на текущото състояние на приложението
    print("Актуализация на състоянието...")
    refresh_ui()
    save_progress()
```


Уверете се, че първо изпълните `az login`, използвайки Azure CLI, за да бъде предоставен правилният контекст за удостоверяване при използване на `DefaultAzureCredential`. Услугата Azure AI Agent не използва API ключове.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Разбира се! Моля, предоставете съдържанието на markdown файла, който искате да бъде преведен.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
